# 🔍 SHAP Explainable AI - Swin Base + MobileViT Hybrid (Optuna)

This notebook applies **SHAP (SHapley Additive exPlanations)** to explain predictions from the **Swin-Base + MobileViT Hybrid model** optimized with Optuna TPE.

## 📊 SHAP Metrics & Visualizations

1. **Mean Absolute SHAP** - Global feature importance across dataset
2. **Summary Plot** - Distribution of pixel/region importance
3. **Force Plot** - Local explanation for individual predictions

## 🎯 Model Details
- **Architecture**: Swin-Base (ImageNet) + MobileViT (Kvasir-pretrained)
- **Classifier**: CatBoost (Optuna-optimized)
- **Feature Dimension**: 1024 (Swin-Base) + 384 (MobileViT) → 256 (fused)

## 📦 Step 1: Install Dependencies

In [ ]:
!pip install shap==0.43.0
!pip install torch torchvision
!pip install catboost
!pip install numpy pandas matplotlib seaborn
!pip install scikit-learn
!pip install pillow
!pip install tqdm

## 📂 Step 2: Upload Project Files

Upload the following files from your local machine:
1. `model.py` - MobileViT architecture
2. `hybrid_model.py` - Hybrid model definition
3. `swin base/optuna tpe swin base/swin_base_kvasir_features.npz` - Extracted features
4. `swin base/optuna tpe swin base/swin_base_kvasir_optuna.cbm` - CatBoost model
5. `mobilevit_kvasir_v2_best_optuna.pth` - MobileViT weights

In [ ]:
from google.colab import files
import os

print("📤 Upload the following files:")
print("1. model.py")
print("2. hybrid_model.py")
print("3. swin_base_kvasir_features.npz")
print("4. swin_base_kvasir_optuna.cbm")
print("5. mobilevit_kvasir_v2_best_optuna.pth")
print("\nClick 'Choose Files' and select all files at once.\n")

uploaded = files.upload()
print(f"\n✅ Uploaded {len(uploaded)} files")

## 🌐 Step 3: Download Kvasir Dataset

In [ ]:
import os
import zipfile
import urllib.request
from tqdm import tqdm

class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_kvasir():
    url = "https://datasets.simula.no/downloads/kvasir/kvasir-dataset-v2.zip"
    output_path = "kvasir-dataset-v2.zip"
    
    if not os.path.exists("kvasir-dataset-v2"):
        print("📥 Downloading Kvasir Dataset...")
        with DownloadProgressBar(unit='B', unit_scale=True, miniters=1, desc=url.split('/')[-1]) as t:
            urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)
        
        print("📦 Extracting dataset...")
        with zipfile.ZipFile(output_path, 'r') as zip_ref:
            zip_ref.extractall()
        
        os.remove(output_path)
        print("✅ Dataset ready!")
    else:
        print("✅ Dataset already exists!")

download_kvasir()

## 🔧 Step 4: Import Libraries and Setup

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import shap
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from hybrid_model import SwinMobileViTHybrid

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Using device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 📊 Step 5: Load Data and Model

In [ ]:
# Load extracted features
print("📂 Loading features...")
data = np.load('swin_base_kvasir_features.npz')
X_train = data['X_train']
X_val = data['X_val']
X_test = data['X_test']
y_train = data['y_train']
y_val = data['y_val']
y_test = data['y_test']

print(f"✅ Features loaded:")
print(f"   Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# Load CatBoost model
print("\n🤖 Loading CatBoost model...")
catboost_model = CatBoostClassifier()
catboost_model.load_model('swin_base_kvasir_optuna.cbm')
print("✅ Model loaded successfully!")

# Class names
class_names = ['dyed-lifted-polyps', 'dyed-resection-margins', 'esophagitis', 
               'normal-cecum', 'normal-pylorus', 'normal-z-line', 
               'polyps', 'ulcerative-colitis']

## 🎯 Step 6: Initialize SHAP Explainer

We'll use **TreeExplainer** for CatBoost, which is optimized for tree-based models.

In [ ]:
print("🔍 Initializing SHAP TreeExplainer...")

# Create SHAP explainer
explainer = shap.TreeExplainer(catboost_model)

# Calculate SHAP values for test set
print("⚙️  Computing SHAP values for test set...")
shap_values = explainer.shap_values(X_test)

print(f"✅ SHAP values computed!")
print(f"   Shape: {shap_values.shape}")
print(f"   Classes: {len(class_names)}")

## 📈 Metric 1: Mean Absolute SHAP Values

**Purpose**: Quantify global feature importance across the entire dataset.

**Key Insight**: Identifies which features (from the 256-dim hybrid representation) have the most significant impact on predictions, regardless of direction.

### Understanding Mean Absolute SHAP for Images:
- Each of the 256 features represents a learned representation from the hybrid Swin-Base + MobileViT model
- Higher values indicate features that consistently influence predictions across the dataset
- These features capture both low-level (texture, color) and high-level (semantic) patterns

In [ ]:
print("📊 Computing Mean Absolute SHAP Values...\n")

# Calculate mean absolute SHAP values across all classes and samples
mean_abs_shap = np.mean(np.abs(shap_values), axis=(0, 1))

# Get top 20 most important features
top_k = 20
top_indices = np.argsort(mean_abs_shap)[-top_k:][::-1]
top_values = mean_abs_shap[top_indices]

# Create DataFrame for better visualization
importance_df = pd.DataFrame({
    'Feature Index': top_indices,
    'Mean Absolute SHAP': top_values,
    'Relative Importance (%)': (top_values / top_values[0] * 100)
})

print("🔝 Top 20 Most Important Features:")
print(importance_df.to_string(index=False))

# Overall statistics
print(f"\n📊 Overall Statistics:")
print(f"   Total Features: {len(mean_abs_shap)}")
print(f"   Mean Absolute SHAP (all features): {np.mean(mean_abs_shap):.6f}")
print(f"   Std Dev: {np.std(mean_abs_shap):.6f}")
print(f"   Max: {np.max(mean_abs_shap):.6f}")
print(f"   Min: {np.min(mean_abs_shap):.6f}")
print(f"   Coefficient of Variation: {(np.std(mean_abs_shap) / np.mean(mean_abs_shap)):.4f}")

# Visualize
plt.figure(figsize=(14, 8))
colors = plt.cm.viridis(np.linspace(0.3, 0.9, top_k))
bars = plt.barh(range(top_k), top_values, color=colors, edgecolor='navy', alpha=0.8)
plt.yticks(range(top_k), [f'Feature {i}' for i in top_indices])
plt.xlabel('Mean Absolute SHAP Value', fontsize=12, fontweight='bold')
plt.ylabel('Feature Index', fontsize=12, fontweight='bold')
plt.title('Top 20 Most Important Features (Mean Absolute SHAP)\nSwin-Base + MobileViT Hybrid', 
          fontsize=14, fontweight='bold', pad=20)
plt.grid(axis='x', alpha=0.3)

# Add value labels
for i, (bar, val) in enumerate(zip(bars, top_values)):
    plt.text(val, bar.get_y() + bar.get_height()/2, f'{val:.4f}', 
             va='center', ha='left', fontsize=8, fontweight='bold')

plt.tight_layout()
plt.savefig('mean_absolute_shap_swin_base.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n💾 Saved: mean_absolute_shap_swin_base.png")

## 📊 Metric 2: SHAP Summary Plot

**Purpose**: Visualize the distribution of feature importance across all samples.

**Key Insight**: Shows not just which features are important, but also:
- The spread of their impact across different samples
- Whether high/low feature values push predictions in a particular direction
- The density and distribution of feature contributions
- Outliers and edge cases in feature behavior

In [ ]:
print("📊 Creating SHAP Summary Plot...\n")

# Summary plot for all classes combined
plt.figure(figsize=(14, 10))
shap.summary_plot(shap_values, X_test, max_display=20, show=False)
plt.title('SHAP Summary Plot - Feature Importance Distribution\nSwin-Base + MobileViT Hybrid (All Classes)', 
          fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('shap_summary_plot_all_classes_swin_base.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Saved: shap_summary_plot_all_classes_swin_base.png\n")

# Create summary plots for each class
print("Creating per-class summary plots...")
for class_idx, class_name in enumerate(class_names):
    plt.figure(figsize=(14, 10))
    shap.summary_plot(shap_values[:, :, class_idx], X_test, max_display=20, show=False)
    plt.title(f'SHAP Summary Plot - {class_name}\nSwin-Base + MobileViT Hybrid', 
              fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    filename = f'shap_summary_{class_name.replace("-", "_")}_swin_base.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"   ✅ Saved: {filename}")

print("\n✅ All summary plots created!")

## 🎯 Metric 3: SHAP Force Plots

**Purpose**: Explain individual predictions in detail.

**Key Insight**: Shows exactly how each feature contributes to moving the prediction from the base value (average prediction) to the actual prediction for a specific sample.

### Interpretation:
- **Red features**: Push prediction higher (increase probability)
- **Blue features**: Push prediction lower (decrease probability)
- **Feature magnitude**: Size of the bar indicates strength of contribution
- **Base value**: Starting point (average model output)
- **Output value**: Final prediction after all feature contributions

In [ ]:
print("🎯 Creating SHAP Force Plots for Sample Predictions...\n")

# Initialize JavaScript visualization
shap.initjs()

# Select representative samples from each class
samples_per_class = 2
selected_samples = []
sample_info = []

for class_idx in range(len(class_names)):
    class_mask = y_test == class_idx
    class_indices = np.where(class_mask)[0]
    
    if len(class_indices) >= samples_per_class:
        # Select samples with highest and lowest confidence
        class_probs = catboost_model.predict_proba(X_test[class_indices])[:, class_idx]
        high_conf_idx = class_indices[np.argmax(class_probs)]
        low_conf_idx = class_indices[np.argmin(class_probs)]
        
        selected_samples.extend([high_conf_idx, low_conf_idx])
        sample_info.append({
            'class': class_names[class_idx],
            'high_conf': np.max(class_probs),
            'low_conf': np.min(class_probs)
        })

print(f"Selected {len(selected_samples)} samples for force plots\n")

# Create force plots
for idx, sample_idx in enumerate(selected_samples[:10]):  # Limit to 10 for notebook
    true_class = y_test[sample_idx]
    pred_probs = catboost_model.predict_proba(X_test[sample_idx:sample_idx+1])[0]
    pred_class = np.argmax(pred_probs)
    confidence = pred_probs[pred_class]
    
    print(f"Sample {idx + 1}:")
    print(f"   True: {class_names[true_class]}")
    print(f"   Predicted: {class_names[pred_class]} (confidence: {confidence:.2%})")
    
    # Force plot for the predicted class
    force_plot = shap.force_plot(
        explainer.expected_value[pred_class],
        shap_values[sample_idx, :, pred_class],
        X_test[sample_idx, :],
        matplotlib=True,
        show=False
    )
    
    match_symbol = "✅" if true_class == pred_class else "❌"
    plt.title(f'Force Plot - Sample {idx + 1} {match_symbol}\nTrue: {class_names[true_class]} | Predicted: {class_names[pred_class]} ({confidence:.1%})',
              fontsize=12, fontweight='bold', pad=10)
    plt.tight_layout()
    filename = f'force_plot_sample_{idx + 1}_swin_base.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"   💾 Saved: {filename}\n")

print("✅ Force plots created!")

## 🌊 Additional: Waterfall Plots

Waterfall plots provide an alternative view of individual predictions, showing the cumulative effect of features in a sequential manner.

In [ ]:
print("🌊 Creating SHAP Waterfall Plots...\n")

# Create waterfall plots for first 5 samples
for idx in range(min(5, len(X_test))):
    true_class = y_test[idx]
    pred_probs = catboost_model.predict_proba(X_test[idx:idx+1])[0]
    pred_class = np.argmax(pred_probs)
    confidence = pred_probs[pred_class]
    
    # Create explanation object
    explanation = shap.Explanation(
        values=shap_values[idx, :, pred_class],
        base_values=explainer.expected_value[pred_class],
        data=X_test[idx, :]
    )
    
    plt.figure(figsize=(12, 8))
    shap.waterfall_plot(explanation, max_display=15, show=False)
    match_symbol = "✅" if true_class == pred_class else "❌"
    plt.title(f'Waterfall Plot - Sample {idx + 1} {match_symbol}\nTrue: {class_names[true_class]} | Predicted: {class_names[pred_class]} ({confidence:.1%})',
              fontsize=12, fontweight='bold', pad=10)
    plt.tight_layout()
    filename = f'waterfall_plot_sample_{idx + 1}_swin_base.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"   💾 Saved: {filename}\n")

print("✅ Waterfall plots created!")

## 📊 Feature Importance Heatmap

In [ ]:
print("🔥 Creating Feature Importance Heatmap...\n")

# Calculate mean absolute SHAP per class
mean_shap_per_class = np.mean(np.abs(shap_values), axis=0)  # Shape: (features, classes)

# Get top 30 features
top_features = 30
overall_importance = np.mean(mean_shap_per_class, axis=1)
top_feature_indices = np.argsort(overall_importance)[-top_features:][::-1]

# Create heatmap data
heatmap_data = mean_shap_per_class[top_feature_indices, :].T

# Plot
plt.figure(figsize=(16, 10))
sns.heatmap(heatmap_data, 
            xticklabels=[f'F{i}' for i in top_feature_indices],
            yticklabels=class_names,
            cmap='YlOrRd',
            cbar_kws={'label': 'Mean Absolute SHAP'},
            linewidths=0.5,
            linecolor='gray',
            annot=False)

plt.title('Feature Importance Heatmap Across Classes\nSwin-Base + MobileViT Hybrid', 
          fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Feature Index', fontsize=12, fontweight='bold')
plt.ylabel('Class', fontsize=12, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('feature_importance_heatmap_swin_base.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Saved: feature_importance_heatmap_swin_base.png")

## 🔬 Class-Specific Feature Analysis

In [ ]:
print("🔬 Analyzing Class-Specific Feature Importance...\n")

# For each class, find its most discriminative features
class_specific_features = {}

for class_idx, class_name in enumerate(class_names):
    # Get SHAP values for this class
    class_shap = np.abs(shap_values[:, :, class_idx])
    mean_class_shap = np.mean(class_shap, axis=0)
    
    # Get top 10 features for this class
    top_10_idx = np.argsort(mean_class_shap)[-10:][::-1]
    top_10_vals = mean_class_shap[top_10_idx]
    
    class_specific_features[class_name] = {
        'indices': top_10_idx,
        'values': top_10_vals
    }
    
    print(f"\n{class_name.upper()}:")
    print("Top 10 Features:")
    for i, (idx, val) in enumerate(zip(top_10_idx, top_10_vals), 1):
        print(f"   {i}. Feature {idx}: {val:.6f}")

print("\n✅ Class-specific analysis complete!")

## 📋 Comprehensive Summary Report

In [ ]:
print("="*80)
print("SHAP EXPLAINABILITY ANALYSIS SUMMARY")
print("Swin-Base + MobileViT Hybrid (Optuna Optimized)")
print("="*80)

print("\n🏗️  MODEL ARCHITECTURE:")
print("   - Backbone 1: Swin-Base (ImageNet-1K) → 1024-dim")
print("   - Backbone 2: MobileViT (Kvasir-pretrained) → 384-dim")
print("   - Fusion: 1408-dim → 256-dim (learned representation)")
print("   - Classifier: CatBoost (Optuna-optimized)")

print("\n📊 METRICS COMPUTED:")
print("\n1. Mean Absolute SHAP Values")
print(f"   - Total features analyzed: {len(mean_abs_shap)}")
print(f"   - Average importance: {np.mean(mean_abs_shap):.6f}")
print(f"   - Top feature importance: {np.max(mean_abs_shap):.6f}")
print(f"   - Most important feature: Feature {np.argmax(mean_abs_shap)}")
print(f"   - Importance concentration (CV): {(np.std(mean_abs_shap) / np.mean(mean_abs_shap)):.4f}")

print("\n2. Summary Plots")
print(f"   - Generated for all {len(class_names)} classes")
print(f"   - Shows distribution of feature contributions")
print(f"   - Visualizes feature value impact on predictions")
print(f"   - Reveals feature interaction patterns")

print("\n3. Force Plots")
print(f"   - Created for {min(10, len(selected_samples))} representative samples")
print(f"   - Shows individual prediction explanations")
print(f"   - Highlights positive/negative feature contributions")
print(f"   - Includes high and low confidence predictions per class")

print("\n4. Additional Visualizations")
print("   - Waterfall plots for detailed feature analysis")
print("   - Heatmap showing class-specific feature importance")
print("   - Class-specific feature rankings")

print("\n📁 OUTPUT FILES:")
output_files = [f for f in os.listdir('.') if f.endswith('.png')]
print(f"   Total visualizations: {len(output_files)}")
print("   Key files:")
print("   - mean_absolute_shap_swin_base.png")
print("   - shap_summary_plot_all_classes_swin_base.png")
print(f"   - {len(class_names)} per-class summary plots")
print(f"   - {min(10, len(selected_samples))} force plots")
print(f"   - {min(5, len(X_test))} waterfall plots")
print("   - feature_importance_heatmap_swin_base.png")

print("\n🎯 MODEL PERFORMANCE:")
test_acc = catboost_model.score(X_test, y_test)
train_acc = catboost_model.score(X_train, y_train)
val_acc = catboost_model.score(X_val, y_val)
print(f"   - Training Accuracy: {train_acc * 100:.2f}%")
print(f"   - Validation Accuracy: {val_acc * 100:.2f}%")
print(f"   - Test Accuracy: {test_acc * 100:.2f}%")
print(f"   - Generalization Gap: {(train_acc - test_acc) * 100:.2f}%")
print(f"   - Test Samples: {len(X_test)}")
print(f"   - Feature Dimension: {X_test.shape[1]}")

print("\n💡 KEY INSIGHTS:")
print(f"   - Feature importance is {'concentrated' if np.std(mean_abs_shap)/np.mean(mean_abs_shap) > 0.5 else 'distributed'}")
print(f"   - Top 20 features account for {np.sum(top_values)/np.sum(mean_abs_shap)*100:.1f}% of total importance")
print(f"   - Model uses {np.sum(mean_abs_shap > np.mean(mean_abs_shap))} features above average importance")

print("\n" + "="*80)
print("✅ SHAP ANALYSIS COMPLETE!")
print("="*80)

## 💾 Download All Results

In [ ]:
import zipfile
from google.colab import files

print("📦 Creating ZIP archive of all SHAP results...\n")

# Create ZIP file
zip_filename = 'SHAP_XAI_Swin_Base_Results.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # Add all PNG files
    png_files = [f for f in os.listdir('.') if f.endswith('.png')]
    for file in png_files:
        zipf.write(file)
        print(f"   Added: {file}")

print(f"\n✅ Created: {zip_filename}")
print(f"   Files included: {len(png_files)}")
print(f"   Size: {os.path.getsize(zip_filename) / 1024 / 1024:.2f} MB")

# Download
print("\n📥 Downloading...")
files.download(zip_filename)
print("✅ Download complete!")